In [1]:
import better_exceptions
import random
import math
from pathlib import Path
from PIL import Image
import numpy as np
import pandas as pd
import cv2 as cv
from keras.utils import Sequence, to_categorical
import Augmentor


def get_transform_func():
    p = Augmentor.Pipeline()
    p.flip_left_right(probability=0.5)
    p.rotate(probability=1, max_left_rotation=5, max_right_rotation=5)
    p.zoom_random(probability=0.5, percentage_area=0.95)
    p.random_distortion(probability=0.5, grid_width=2, grid_height=2, magnitude=8)
    p.random_color(probability=1, min_factor=0.8, max_factor=1.2)
    p.random_contrast(probability=1, min_factor=0.8, max_factor=1.2)
    p.random_brightness(probability=1, min_factor=0.8, max_factor=1.2)
    p.random_erasing(probability=0.5, rectangle_area=0.2)

    def transform_image(image):
        image = [Image.fromarray(image)]
        for operation in p.operations:
            r = round(random.uniform(0, 1), 1)
            if r <= operation.probability:
                image = operation.perform_operation(image)
        return image[0]
    return transform_image


class FaceGenerator(Sequence):
    def __init__(self, appa_dir, utk_dir=None, batch_size=32, image_size=224):
        self.image_path_and_age = []
        self._load_appa(appa_dir)

        if utk_dir:
            self._load_utk(utk_dir)

        self.image_num = len(self.image_path_and_age)
        self.batch_size = batch_size
        self.image_size = image_size
        self.indices = np.random.permutation(self.image_num)
        self.transform_image = get_transform_func()

    def __len__(self):
        return self.image_num // self.batch_size

    def __getitem__(self, idx):
        batch_size = self.batch_size
        image_size = self.image_size
        x = np.zeros((batch_size, image_size, image_size, 3), dtype=np.uint8)
        y = np.zeros((batch_size, 1), dtype=np.int32)

        sample_indices = self.indices[idx * batch_size:(idx + 1) * batch_size]

        for i, sample_id in enumerate(sample_indices):
            image_path, age = self.image_path_and_age[sample_id]
            image = cv2.imread(str(image_path))
            x[i] = self.transform_image(cv2.resize(image, (image_size, image_size)))
            age += math.floor(np.random.randn() * 2 + 0.5)
            y[i] = np.clip(age, 0, 100)

        return x, to_categorical(y, 101)

    def on_epoch_end(self):
        self.indices = np.random.permutation(self.image_num)

    def _load_appa(self, appa_dir):
        appa_root = Path(appa_dir)
        train_image_dir = appa_root.joinpath("train")
        gt_train_path = appa_root.joinpath("gt_avg_train.csv")
        df = pd.read_csv(str(gt_train_path))

        for i, row in df.iterrows():
            age = min(100, int(row.apparent_age_avg))
            # age = int(row.real_age)
            image_path = train_image_dir.joinpath(row.file_name + "_face.jpg")

            if image_path.is_file():
                self.image_path_and_age.append([str(image_path), age])

    def _load_utk(self, utk_dir):
        image_dir = Path(utk_dir)

        for image_path in image_dir.glob("*.jpg"):
            image_name = image_path.name  # [age]_[gender]_[race]_[date&time].jpg
            age = min(100, int(image_name.split("_")[0]))

            if image_path.is_file():
                self.image_path_and_age.append([str(image_path), age])


class ValGenerator(Sequence):
    def __init__(self, appa_dir, batch_size=32, image_size=224):
        self.image_path_and_age = []
        self._load_appa(appa_dir)
        self.image_num = len(self.image_path_and_age)
        self.batch_size = batch_size
        self.image_size = image_size

    def __len__(self):
        return self.image_num // self.batch_size

    def __getitem__(self, idx):
        batch_size = self.batch_size
        image_size = self.image_size
        x = np.zeros((batch_size, image_size, image_size, 3), dtype=np.uint8)
        y = np.zeros((batch_size, 1), dtype=np.int32)

        for i in range(batch_size):
            image_path, age = self.image_path_and_age[idx * batch_size + i]
            image = cv2.imread(str(image_path))
            x[i] = cv2.resize(image, (image_size, image_size))
            y[i] = age

        return x, to_categorical(y, 101)

    def _load_appa(self, appa_dir):
        appa_root = Path(appa_dir)
        val_image_dir = appa_root.joinpath("valid")
        gt_val_path = appa_root.joinpath("gt_avg_valid.csv")
        df = pd.read_csv(str(gt_val_path))

        for i, row in df.iterrows():
            age = min(100, int(row.apparent_age_avg))
            # age = int(row.real_age)
            image_path = val_image_dir.joinpath(row.file_name + "_face.jpg")

            if image_path.is_file():
                self.image_path_and_age.append([str(image_path), age])

Using TensorFlow backend.
/home/bigred/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/bigred/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/bigred/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/bigred/.local/lib/python3.6/site-packages/tensorflow/python

# CV2
   * 套件與模組導入
   * 功能實現
   * 結束並釋放資源
   * [OPENCV基礎2](https://zhuanlan.zhihu.com/p/32904337)

## 導入模組

In [2]:
import cv2 as cv 
import numpy as np

## 讀取圖片檔 (檔名,引數)

### 圖像矩陣輸出

In [3]:

img=cv.imread("123.jpeg",2) ###flag=-1時，8位深度，原通道 flag=0，8位深度，1通道 flag=1, 8位深度 ，3通道 flag=2，原深度，1通道 flag=3, 原深度，3通道 flag=4，8位深度 ，3通道###
print(img)

[[ 34  34  34 ... 154 154 155]
 [ 34  34  34 ... 154 154 155]
 [ 34  34  34 ... 153 153 154]
 ...
 [ 40  40  40 ...  47  47  53]
 [ 44  44  44 ...  47  47  47]
 [ 46  46  46 ...  47  47  46]]


### 圖像輸出

In [4]:
print(img.shape)
print(img.dtype)
print(img.min())
print(img.max())
img_reshape=img.reshape(-1,1)
print(img_reshape)

#建立視窗並顯示影象
cv.namedWindow("img")
cv.imshow("img",img)
cv.waitKey(0)

(157, 321)
uint8
0
255
[[34]
 [34]
 [34]
 ...
 [47]
 [47]
 [46]]


226

In [5]:
#釋放視窗
cv.destroyAllWindows() 

## 劃線

In [6]:
img=cv.line(img,(0,0),(511,511),(255,0,0),5)

In [ ]:
cv.namedWindow("img")
cv.imshow("img",img)
cv.waitKey(0)

In [ ]:
#釋放視窗
cv.destroyAllWindows() 

## 輔助線

In [5]:
# img=cv.rectangle(img,pt1,pt2,color,lineType)
img=cv.rectangle(img,(384,0),(510,128),(0,255,0),3)

In [ ]:
cv.namedWindow("img")
cv.imshow("img",img)
cv.waitKey(0)

In [ ]:
#釋放視窗
cv.destroyAllWindows() 

## 畫圓圈

In [6]:
img=cv.circle(img,(447,63),63,(0,0,255),-1)

## 讀取Louisa影片

[影像轉圖片](https://blog.csdn.net/m0_37733057/article/details/79023693)

In [3]:
import cv2
vc = cv2.VideoCapture('20190410.avi') #读入视频文件
c=0
rval=vc.isOpened()
#timeF = 1  #视频帧计数间隔频率
while rval:   #循环读取视频帧
    c = c + 1
    rval, frame = vc.read()
#    if(c%timeF == 0): #每隔timeF帧进行存储操作
#        cv2.imwrite('smallVideo/smallVideo'+str(c) + '.jpg', frame) #存储为图像
    if rval:
        cv2.imwrite('20190410/20190410'+str(c).zfill(8) + '.jpg', frame) #存储为图像
        cv2.waitKey(1)
    else:
        break
vc.release()


In [4]:
# 检测出图片中的人脸，并用方框标记出来
    def face_detector(image, cascade):
        global face_num #引用全局变量
        grayImage = cv.cvtColor(image, cv.COLOR_BGR2GRAY) #灰度化图片 
        equalImage = cv.equalizeHist(grayImage) #直方图均衡化
        faces = cascade.detectMultiScale(equalImage, scaleFactor=1.3, minNeighbors=3)
     
        for (x,y,w,h) in faces:
            #裁剪出人脸，单独保存成图片，注意这里的横坐标与纵坐标不知为啥颠倒了
            #cv.imwrite("face_%s.png" %(face_num), image[y:y+h,x:x+w])
            cv.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
            face_num = face_num + 1
        return image
